# Importing llamas_pyjamas modules

**Please follow the installation instructions prior to attempting this**

In [ ]:
import os
import sys
import numpy as np
import pickle
import ray
import pkg_resources
import glob
import traceback
from   pathlib import Path
from   llamas_pyjamas.config import BASE_DIR, OUTPUT_DIR, DATA_DIR

# Get package root and add to path before other imports as a precaution -> if installed as package this should hopefully not be needed
package_root = Path().absolute().parent
sys.path.append(str(package_root))
sys.path.append(BASE_DIR+'/')


ray.init(ignore_reinit_error=True)
import llamas_pyjamas.Trace.traceLlamasMulti as trace # type: ignore
import llamas_pyjamas.Extract.extractLlamas as extract # type: ignore
from llamas_pyjamas.Image.WhiteLight import WhiteLight, WhiteLightFits

In [ ]:
# Get absolute path to llamas_pyjamas package to check the installation
package_path = pkg_resources.resource_filename('llamas_pyjamas', '')
package_root = os.path.dirname(package_path)

print(f"Package path: {package_path}")
print(f"Package root: {package_root}")

**Optional Ray initialisation if attempting to parallelise some of the processes**

In [ ]:
# Configure Ray runtime environment
runtime_env = {
    "py_modules": [package_root],
    "env_vars": {"PYTHONPATH": f"{package_root}:{os.environ.get('PYTHONPATH', '')}"},
    "excludes": [
        str(Path(DATA_DIR) / "**"),  # Exclude DATA_DIR and all subdirectories
        "**/*.fits",                 # Exclude all FITS files anywhere
        "**/*.gz",                 # Exclude all tarballs files anywhere
        "**/*.zip",                 # Exclude all zip files anywhere
        "**/*.pkl",                  # Exclude all pickle files anywhere
        "**/.git/**",               # Exclude git directory
    ]
}

# Initialize Ray
ray.shutdown()
ray.init(runtime_env=runtime_env)

In [ ]:
##This is also optional, but it can help with debugging
DATA_DIR = "/Users/simcoe/Science/LLAMAS/CommissioningRun/NIGHT5"
DATA_DIR = "/Users/simcoe/Science/LLAMAS/RUN02/2025-03-05"
import importlib
importlib.reload(trace)

# Extraction

To run the extraction process on a single fits image and then plot a single spectrum:

In [ ]:
from llamas_pyjamas.GUI.guiExtract import GUI_extract
filepath = 'Path_to_your_fits_file'
# Example: filepath = '/Users/slh/Documents/Projects/Magellan_dev/LLAMAS/comissioning_data/20241128/LLAMAS_2024-11-28T03_50_39.584_mef.fits

#This should take a few mintues to run and in your output folder you should see the extracted pickle file
GUI_extract(filepath)


## General code for picking a fibre to plot the spectrum of

In [ ]:

#Example: extract_pickle = '/Users/slh/Documents/Projects/Magellan_dev/LLAMAS/llamas-pyjamas/llamas_pyjamas/output/LLAMASExtract_batch_20250203_215658.pkl'
extract_pickle = 'output/your_extraction_file.pkl'


with open(extract_pickle, 'rb') as f:
    exobj = pickle.load(f)

extraction_list = exobj['extractions']

#select which extraction object you wish to plot
# Each object represents a single HDU extension in the FITS file which was extracted. 
# The order should correspond to the order in the fits file 
# (but don't forget that the fits file has a primary HDU as well so the first detector image is HDU 1 in the fits but index 0 in the extraction list)

HDU_idx = 0 #change the '0' to the value of the HDU you wish to plot spectra from
spec_arrays = extraction_list[HDU_idx].counts #change the '0' to the value of the HDU you wish to plot spectra from

#This gives you an array of the extracted spectra for the selected HDU, shape = (Nfib, 2048)
#You can plot the spectra for a single fiber using the following code

import matplotlib.pyplot as plt
Nfib = 0 #select the fiber you wish to plot
plt.plot(spec_arrays[Nfib])
plt.show()


### To try and find which fibre the object is in:

In [ ]:
idx = [0, 3, 6, 9, 12, 15, 18, 21] ### These are the indexes which represent the red detector outputs, 1A, 1B, 2A, 2B, 3A, 3B, 4A, 4B
                                ## If you want the green detector outputs +1 to each index, for blue +2 for each index


## This code will go through each detector output and print the maximum summed value and the row with the maximum summed value
#think of this as trying to find the fibre your object is most likely to be on
for i in idx:
    data = exobj['extractions'][i].counts
    row_sums = np.nansum(data, axis=1)
    max_sum = np.nanmax(row_sums)
    row_with_max_sum = np.argmax(row_sums)
    print(i, max_sum, row_with_max_sum)

idx_with_obj = 0 #select the index of the detector output you wish to extract the spectra from    
row_with_object = 0 #select the row with the object you wish to extract based on what the loop prints out
#You should pick the the idx and row where the maximum sum is highest, this is the most likely fibre the object is on      
spectra = exobj['extractions'][idx_with_obj].counts[row_with_max_sum]

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
plt.plot(spectra)
plt.ylabel('Counts')
plt.show()

# If you want to extract a specific detector rather than the whole IFU image:

In [ ]:
from llamas_pyjamas.Extract.extractLlamas import ExtractLlamas
import pickle
from astropy.io import fits

### MAKE SURE THE COLOUR AND BENCHSIDE OF THE TRACEFILE YOU LOAD IN MATCHES THE HDU YOU WANT TO EXTRACT
#e.g. red_1A typically corresponds to hdu[1] in the fits file
#To check this print hdu[i].header
with open(DATA_DIR + '/Your_trace_file.pkl', 'rb') as f:
    traceobj = pickle.load(f)

hdu = fits.open('Path_to_your_fits_file.fits')
ex = ExtractLlamas(traceobj, hdu[hdu_index].data.astype(float), dict(hdu[hdu_index].header))

## Extract the flat field (for getting fiber throughputs later)

In [ ]:
# from llamas_pyjamas.GUI.guiExtract import GUI_extract
import llamas_pyjamas.GUI.guiExtract as ge
importlib.reload(ge)

skyflat_filename = os.path.join(DATA_DIR, 'LLAMAS_2025-03-05T23_24_13.255_mef.fits')
# trace_filename = os.path.join(DATA_DIR, 'LLAMAS_2024-11-29T23_50_11.041_mef.fits')
trace_filename = os.path.join(DATA_DIR, 'LLAMAS_2025-03-05T23_24_13.255_mef.fits')
ge.GUI_extract(skyflat_filename)

In [ ]:
importlib.reload(extract)
flat_picklename = os.path.join(OUTPUT_DIR, 'LLAMAS_2024-11-29T23_50_11.041_extract.pkl')
flatdict = extract.ExtractLlamas.loadExtraction(flat_picklename)
# dir(flatdict['extractions'][18])

In [ ]:
flatdict['extractions'][10].relative_throughput[5] = 0.96
flatdict['extractions'][10].relative_throughput

## Extract an arc frame

In [ ]:
arc_filename   = os.path.join(DATA_DIR, 'LLAMAS_2025-03-05T19_04_46.449_mef.fits')
ge.GUI_extract(arc_filename)

### Read in and plot up arc extractions

In [ ]:
importlib.reload(extract)
import pickle
arc_filename   = os.path.join(DATA_DIR, 'LLAMAS_2025-03-05T19_04_46.449_mef.fits')
arc_picklename = os.path.join(OUTPUT_DIR, os.path.basename(arc_filename).replace('_mef.fits', '_extract.pkl'))
with open(arc_picklename, 'rb') as fp:
    batch_data = pickle.load(fp)
arcdict = extract.ExtractLlamas.loadExtraction(arc_picklename)
arcspec, metadata = arcdict['extractions'], arcdict['metadata']

In [ ]:
import llamas_pyjamas.Arc.arcLlamas as arc
importlib.reload(arc)
importlib.reload(extract)
arc.shiftArcX(arc_picklename)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib qt
shift_picklename = arc_picklename.replace('_extract.pkl', '_extract_shifted.pkl')
#arcspec_shifted, metadata_shifted = extract.load_extractions(shift_picklename)
#[plt.plot(arcspec_shifted[18].xshift[i,:], arcspec_shifted[18].counts[i,:],".") for i in range(100,200)]
#plt.show()

In [ ]:
importlib.reload(arc)
#flatdict = extract.ExtractLlamas.loadExtraction(flat_picklename)
#flatdict['extractions'][10].relative_throughput[10] = 100
#flatdict['extractions'][10].relative_throughput
# arcdict = extract.ExtractLlamas.loadExtraction(shift_picklename)
# gd = (arcdict['extractions'][18].xshift[150] > 100)
# plt.plot(flatdict['extractions'][18].counts[150][gd])
# plt.show()

arc.fiberRelativeThroughput(flat_picklename, shift_picklename)

In [ ]:
ray.shutdown()

In [ ]:
tp=shift_picklename.replace('.pkl','_shifted_tp.pkl')
importlib.reload(arc)
arc.arcSolve(tp)

In [ ]:
arcdict = extract.ExtractLlamas.loadExtraction(os.path.join(OUTPUT_DIR, 'LLAMAS_reference_arc.pkl'))

Now extract a standard star spectrum

In [ ]:
import llamas_pyjamas.GUI.guiExtract as ge
importlib.reload(ge)
std_filename   = os.path.join(DATA_DIR, 'LLAMAS_2025-03-06T00_15_52.794_mef.fits')
ge.GUI_extract(std_filename, method='boxcar')

Load the extracted standard

In [ ]:
std_dict = extract.ExtractLlamas.loadExtraction(os.path.join(OUTPUT_DIR,'LLAMAS_2025-03-06T00_15_52.794_extract.pkl'))
std_extract = std_dict['extractions']

In [ ]:
import llamas_pyjamas.Arc.arcLlamas as arc
importlib.reload(arc)
std_wvcal = arc.arcTransfer(std_dict, arcdict)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib qt
import Flux.calcThroughput as flux
importlib.reload(flux)
importlib.reload(arc)
flux.calcThroughput(std_dict, arcdict, 'blue')
flux.calcThroughput(std_dict, arcdict, 'green')
flux.calcThroughput(std_dict, arcdict, 'red')
plt.ylim(0,40)
plt.xlabel('Wavelength (Angstrom)')
plt.ylabel('Relative Throughput (%)')
#plt.show()